In [2]:
import os
os.chdir('/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/src/vendor_run/mellotron')
from hparams import create_hparams
from model import Tacotron2
import torch
from loss_function import Tacotron2Loss
from train import prepare_dataloaders
import waveglow.glow
from waveglow.denoiser import Denoiser
import IPython.display as ipd
import sys
from text import cmudict, text_to_sequence
from data_utils import TextMelLoader, TextMelCollate
import waveglow
import librosa
from layers import TacotronSTFT
from train import warm_start_model
from torch.utils.data import DataLoader
from train import save_checkpoint, load_checkpoint

sys.path.append('waveglow/')
def load_mel(path):
    audio, sampling_rate = librosa.core.load(path, sr=hparams.sampling_rate)
    audio = torch.from_numpy(audio)
    if sampling_rate != hparams.sampling_rate:
        raise ValueError("{} SR doesn't match target {} SR".format(
            sampling_rate, stft.sampling_rate))
    audio_norm = audio.unsqueeze(0)
    audio_norm = torch.autograd.Variable(audio_norm, requires_grad=False)
    melspec = stft.mel_spectrogram(audio_norm)
    melspec = melspec.cuda()
    return melspec


def load_model(hparams):
    model = Tacotron2(hparams).cuda()
    if hparams.fp16_run:
        model.decoder.attention_layer.score_mask_value = finfo('float16').min

    return model

arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
waveglow_path = '../../vendor_tools/waveglow/waveglow_256channels_universal_v4.pt'
waveglow = torch.load(waveglow_path)['model'].cuda().eval()
denoiser = Denoiser(waveglow).cuda().eval()
#from glow import infer

hparams = create_hparams(None)
hparams.n_speakers = 3
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/uber3_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/uber3_val.txt'
hparams.batch_size = 16
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/uber3/checkpoint_9500'
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/ljmel/checkpoint_8000'

hparams.warm_start = True
learning_rate = 1e-3#

stft = TacotronSTFT(hparams.filter_length, hparams.hop_length, hparams.win_length,
                    hparams.n_mel_channels, hparams.sampling_rate, hparams.mel_fmin,
                    hparams.mel_fmax)

criterion = Tacotron2Loss()
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
iteration = 0
epoch_offset = 0
is_overflow = False

#i, batch = next(enumerate(train_loader))


torch.manual_seed(hparams.seed)
model = load_model(hparams)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
optimizer.eps = 1e-16
#train
# model = warm_start_model(
#     hparams.checkpoint_path, model, hparams.ignore_layers)
model, optimizer, _learning_rate, iteration = load_checkpoint(hparams.checkpoint_path, model, optimizer)




model.eval()
with torch.no_grad():
    val_sampler = None
    batch_size = 4
    val_loader = DataLoader(valset, sampler=val_sampler, num_workers=1,
                            shuffle=False, batch_size=batch_size,
                            pin_memory=False, 
                            collate_fn=collate_fn)

    val_loss = 0.0
    i, batch = next(enumerate(val_loader))
    x, y = model.parse_batch(batch)
    y_pred = model(x)
    loss = criterion(y_pred, y)

Loading checkpoint '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/uber3/checkpoint_9500'
Loaded checkpoint '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/uber3/checkpoint_9500' from iteration 9500


/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/src/vendor_run/mellotron/yin.py:44: RuntimeWarning: invalid value encountered in true_divide
  cmndf = df[1:] * range(1, N) / np.cumsum(df[1:]).astype(float) #scipy method


here


In [9]:
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
audio_paths = hparams.training_files#'data/examples_filelist.txt'
dataloader = TextMelLoader(audio_paths, hparams)
datacollate = TextMelCollate(1)
#file_idx = [0,1,2,3]
file_idx = 0
audio_path, text, sid = dataloader.audiopaths_and_text[0]
text = "its like this and like that and like this and uh"
def to_arpabet(text):
    from g2p_en import G2p
    g2p = G2p()
    return " ".join(
        [
            f"{{ {s.strip()} }}" if s.strip() not in ",." else s.strip()
            for s in " ".join(g2p(text)).split("  ")
        ]
    )
text = to_arpabet(text)
# get audio path, encoded text, pitch contour and mel for gst
text_encoded = torch.LongTensor(text_to_sequence(text, hparams.text_cleaners, arpabet_dict))[None, :].cuda()    
pitch_contour = dataloader[file_idx][3][None].cuda()
mel = load_mel(audio_path)
#print(audio_path, text)


#text, style_input, speaker_ids, f0s
#xnew = (text_encoded, style, x[5][[0]],x[6][[0]])
#xnew = (text_encoded, embedded_gst, x[5][[0]],x[6][[0]])
speaker_ids = torch.tensor([0], device='cuda:0').to(torch.int64)
#xnew = (text_encoded, 0,speaker_ids,x[6][[0]])
xnew =(text_encoded, 0,speaker_ids,torch.zeros([1, 1, 1500], device =  'cuda:0'))
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    rhythm = rhythm.permute(1, 0, 2)
    audio = denoiser(waveglow.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]

In [10]:
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [6]:
x[6][[0]].shape

torch.Size([1, 1, 481])

In [24]:
speaker_ids = torch.tensor([1], device='cuda:0').to(torch.int64)
#xnew = (text_encoded, 0,speaker_ids,x[6][[0]])
xnew =(text_encoded, 0,speaker_ids,torch.zeros([1, 1, 481], device =  'cuda:0'))
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    rhythm = rhythm.permute(1, 0, 2)
    audio = denoiser(waveglow.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

Warning! Reached max decoder steps


In [25]:
speaker_ids = torch.tensor([2], device='cuda:0').to(torch.int64)
#xnew = (text_encoded, 0,speaker_ids,x[6][[0]])
xnew =(text_encoded, 0,speaker_ids,torch.zeros([1, 1, 481], device =  'cuda:0'))
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    rhythm = rhythm.permute(1, 0, 2)
    audio = denoiser(waveglow.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

Warning! Reached max decoder steps


In [1]:
import os
os.chdir('/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/src/vendor_run/mellotron')
from hparams import create_hparams
from model import Tacotron2
import torch
from loss_function import Tacotron2Loss
from train import prepare_dataloaders
import waveglow.glow
from waveglow.denoiser import Denoiser
import IPython.display as ipd
import sys
from text import cmudict, text_to_sequence
from data_utils import TextMelLoader, TextMelCollate
import waveglow
import librosa
from layers import TacotronSTFT
from train import warm_start_model
from torch.utils.data import DataLoader
from train import save_checkpoint, load_checkpoint

sys.path.append('waveglow/')
def load_mel(path):
    audio, sampling_rate = librosa.core.load(path, sr=hparams.sampling_rate)
    audio = torch.from_numpy(audio)
    if sampling_rate != hparams.sampling_rate:
        raise ValueError("{} SR doesn't match target {} SR".format(
            sampling_rate, stft.sampling_rate))
    audio_norm = audio.unsqueeze(0)
    audio_norm = torch.autograd.Variable(audio_norm, requires_grad=False)
    melspec = stft.mel_spectrogram(audio_norm)
    melspec = melspec.cuda()
    return melspec


def load_model(hparams):
    model = Tacotron2(hparams).cuda()
    if hparams.fp16_run:
        model.decoder.attention_layer.score_mask_value = finfo('float16').min

    return model

arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
waveglow_path = '../../vendor_tools/waveglow/waveglow_256channels_universal_v4.pt'
waveglow = torch.load(waveglow_path)['model'].cuda().eval()
denoiser = Denoiser(waveglow).cuda().eval()
#from glow import infer

hparams = create_hparams(None)
hparams.n_speakers = 3
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/uber3_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/uber3_val.txt'
hparams.batch_size = 16
hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
hparams.warm_start = True
learning_rate = 1e-3#

stft = TacotronSTFT(hparams.filter_length, hparams.hop_length, hparams.win_length,
                    hparams.n_mel_channels, hparams.sampling_rate, hparams.mel_fmin,
                    hparams.mel_fmax)

criterion = Tacotron2Loss()
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
iteration = 0
epoch_offset = 0
is_overflow = False

#i, batch = next(enumerate(train_loader))


torch.manual_seed(hparams.seed)
model = load_model(hparams)

#train
model = warm_start_model(
    hparams.checkpoint_path, model, hparams.ignore_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
optimizer.eps = 1e-16


model.eval()
with torch.no_grad():
    val_sampler = None
    batch_size = 4
    val_loader = DataLoader(valset, sampler=val_sampler, num_workers=1,
                            shuffle=False, batch_size=batch_size,
                            pin_memory=False, 
                            collate_fn=collate_fn)

    val_loss = 0.0
    i, batch = next(enumerate(val_loader))
    x, y = model.parse_batch(batch)
    y_pred = model(x)
    loss = criterion(y_pred, y)

/opt/conda/lib/python3.7/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv1d' has changed. you can retrieve the original source code by accessing the object's source attribute or

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'


/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/src/vendor_run/mellotron/yin.py:44: RuntimeWarning: invalid value encountered in true_divide
  cmndf = df[1:] * range(1, N) / np.cumsum(df[1:]).astype(float) #scipy method


here


In [2]:
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
audio_paths = hparams.validation_files#'data/examples_filelist.txt'
dataloader = TextMelLoader(audio_paths, hparams)
datacollate = TextMelCollate(1)
#file_idx = [0,1,2,3]
file_idx = 0
audio_path, text, sid = dataloader.audiopaths_and_text[0]
text = "its like this and like that and like this and uh"
def to_arpabet(text):
    from g2p_en import G2p
    g2p = G2p()
    return " ".join(
        [
            f"{{ {s.strip()} }}" if s.strip() not in ",." else s.strip()
            for s in " ".join(g2p(text)).split("  ")
        ]
    )
text = to_arpabet(text)
# get audio path, encoded text, pitch contour and mel for gst
text_encoded = torch.LongTensor(text_to_sequence(text, hparams.text_cleaners, arpabet_dict))[None, :].cuda()    
pitch_contour = dataloader[file_idx][3][None].cuda()
mel = load_mel(audio_path)
#print(audio_path, text)


#text, style_input, speaker_ids, f0s
#xnew = (text_encoded, style, x[5][[0]],x[6][[0]])
#xnew = (text_encoded, embedded_gst, x[5][[0]],x[6][[0]])
speaker_ids = torch.tensor([0], device='cuda:0').to(torch.int64)
#xnew = (text_encoded, 0,speaker_ids,x[6][[0]])
xnew =(text_encoded, 0,speaker_ids,torch.zeros([1, 1, 481], device =  'cuda:0'))
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    rhythm = rhythm.permute(1, 0, 2)
    audio = denoiser(waveglow.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]

In [3]:
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

In [16]:
from numba import cuda 
cuda.select_device(0)
cuda.close()
torch.cuda.empty_cache()

In [7]:
import os
os.chdir('/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/src/vendor_run/mellotron')
from hparams import create_hparams
from model import Tacotron2
import torch
from loss_function import Tacotron2Loss
from train import prepare_dataloaders
import waveglow.glow
from waveglow.denoiser import Denoiser
import IPython.display as ipd
import sys
from text import cmudict, text_to_sequence
from data_utils import TextMelLoader, TextMelCollate
import waveglow
import librosa
from layers import TacotronSTFT
from train import warm_start_model
from torch.utils.data import DataLoader
from train import save_checkpoint, load_checkpoint

sys.path.append('waveglow/')
def load_mel(path):
    audio, sampling_rate = librosa.core.load(path, sr=hparams.sampling_rate)
    audio = torch.from_numpy(audio)
    if sampling_rate != hparams.sampling_rate:
        raise ValueError("{} SR doesn't match target {} SR".format(
            sampling_rate, stft.sampling_rate))
    audio_norm = audio.unsqueeze(0)
    audio_norm = torch.autograd.Variable(audio_norm, requires_grad=False)
    melspec = stft.mel_spectrogram(audio_norm)
    melspec = melspec.cuda()
    return melspec


def load_model(hparams):
    model = Tacotron2(hparams).cuda()
    if hparams.fp16_run:
        model.decoder.attention_layer.score_mask_value = finfo('float16').min

    return model

arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
waveglow_path = '../../vendor_tools/waveglow/waveglow_256channels_universal_v4.pt'
waveglow = torch.load(waveglow_path)['model'].cuda().eval()
denoiser = Denoiser(waveglow).cuda().eval()
#from glow import infer

hparams = create_hparams(None)
hparams.n_speakers = 3
hparams.training_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/uber3_train.txt'
hparams.validation_files = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/filelists/uber3_val.txt'
hparams.batch_size = 16
#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/ljmel/checkpoint_500'

#hparams.checkpoint_path = '/mnt/disks/uberduck-experiments-v0/models/mellotron/vendor/mellotron_ljs.pt'
hparams.warm_start = True
learning_rate = 1e-3#

stft = TacotronSTFT(hparams.filter_length, hparams.hop_length, hparams.win_length,
                    hparams.n_mel_channels, hparams.sampling_rate, hparams.mel_fmin,
                    hparams.mel_fmax)

criterion = Tacotron2Loss()
train_loader, valset, collate_fn, train_sampler = prepare_dataloaders(hparams)
iteration = 0
epoch_offset = 0
is_overflow = False

#i, batch = next(enumerate(train_loader))


torch.manual_seed(hparams.seed)
model = load_model(hparams)

#train
model = warm_start_model(
    hparams.checkpoint_path, model, hparams.ignore_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=hparams.weight_decay)
optimizer.eps = 1e-16


model.eval()
with torch.no_grad():
    val_sampler = None
    batch_size = 4
    val_loader = DataLoader(valset, sampler=val_sampler, num_workers=1,
                            shuffle=False, batch_size=batch_size,
                            pin_memory=False, 
                            collate_fn=collate_fn)

    val_loss = 0.0
    i, batch = next(enumerate(val_loader))
    x, y = model.parse_batch(batch)
    y_pred = model(x)
    loss = criterion(y_pred, y)

Warm starting model from checkpoint '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/outputs/ljmel/checkpoint_500'


/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/src/vendor_run/mellotron/yin.py:44: RuntimeWarning: invalid value encountered in true_divide
  cmndf = df[1:] * range(1, N) / np.cumsum(df[1:]).astype(float) #scipy method


here


In [8]:
arpabet_dict = cmudict.CMUDict('data/cmu_dictionary')
audio_paths = hparams.validation_files#'data/examples_filelist.txt'
dataloader = TextMelLoader(audio_paths, hparams)
datacollate = TextMelCollate(1)
#file_idx = [0,1,2,3]
file_idx = 0
audio_path, text, sid = dataloader.audiopaths_and_text[0]
text = "its like this and like that and like this and uh"
def to_arpabet(text):
    from g2p_en import G2p
    g2p = G2p()
    return " ".join(
        [
            f"{{ {s.strip()} }}" if s.strip() not in ",." else s.strip()
            for s in " ".join(g2p(text)).split("  ")
        ]
    )
text = to_arpabet(text)
# get audio path, encoded text, pitch contour and mel for gst
text_encoded = torch.LongTensor(text_to_sequence(text, hparams.text_cleaners, arpabet_dict))[None, :].cuda()    
pitch_contour = dataloader[file_idx][3][None].cuda()
mel = load_mel(audio_path)
#print(audio_path, text)


#text, style_input, speaker_ids, f0s
#xnew = (text_encoded, style, x[5][[0]],x[6][[0]])
#xnew = (text_encoded, embedded_gst, x[5][[0]],x[6][[0]])
speaker_ids = torch.tensor([0], device='cuda:0').to(torch.int64)
#xnew = (text_encoded, 0,speaker_ids,x[6][[0]])
xnew =(text_encoded, 0,speaker_ids,torch.zeros([1, 1, 481], device =  'cuda:0'))
with torch.no_grad():
    model.zero_grad()
    mel_outputs, mel_outputs_postnet, gate_outputs, rhythm = model.inference(xnew)
    rhythm = rhythm.permute(1, 0, 2)
    audio = denoiser(waveglow.infer(mel_outputs_postnet, sigma=0.8), 0.01)[:, 0]

Warning! Reached max decoder steps


In [9]:
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)